<h2><b><i>Yes Bank Data\thon

In [1]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt # Visualization
import seaborn as sns # For easier statistical plotting
sns.set_style("whitegrid")

In [2]:
train = pd.read_csv('Yes_Bank_Training.csv')

In [3]:
#Process Labels into binary
def process_label(text):
    if text == 'yes':
        return 1
    else:
        return 0

In [4]:
'''
data = train
cat_vars = ['job_description','marital_status','education_details','has_default','housing_status','previous_loan','phone_type','month_of_year','poutcome_of_campaign']
for var in cat_vars:
    cat_list = 'var' + '_' + var
    cat_list = pd.get_dummies(train[var], prefix=var)
    data1 = pd.concat([data,cat_list], axis=1)
    data = data1

cat_vars = ['job_description','marital_status','education_details','has_default','housing_status','previous_loan','phone_type','month_of_year','poutcome_of_campaign']
data_vars = data.columns.values.tolist()
to_keep = [i for i in data_vars if i not in cat_vars]
'''
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in train.columns.values:
           if train[col].dtypes=='object':
            data=train[col]
            le.fit(data.values)
            train[col]=le.transform(train[col])

In [6]:
# Train = data[to_keep]
trainset = train
trainset.drop('serial_number',axis=1,inplace=True)

In [9]:
trainset.describe()

,age_in_years,job_description,marital_status,education_details,has_default,balance_in_account,housing_status,previous_loan,phone_type,date,month_of_year,call_duration,campaign_contacts,days_passed,previous_contact,poutcome_of_campaign,outcome
count,31649.000000,31649.000000,31649.000000,31649.00000,31649.000000,31649.000000,31649.000000,31649.000000,31649.000000,31649.000000,31649.000000,31649.000000,31649.000000,31649.000000,31649.000000,31649.000000,31649.000000
mean,41.111820,4.371607,1.134823,1.21056,0.022876,1293.382540,0.544409,0.183323,0.860470,16.597997,5.507062,252.408196,3.062372,10.819931,0.208537,2.827040,0.058138
std,9.597652,3.286769,0.597267,0.74909,0.149510,2961.185138,0.498032,0.386937,0.962679,8.586414,2.679529,262.344981,3.510245,48.086849,1.946336,0.670717,0.234007
min,19.000000,0.000000,0.000000,0.00000,0.000000,-8019.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000
25%,33.000000,1.000000,1.000000,1.00000,0.000000,47.000000,0.000000,0.000000,0.000000,9.000000,4.000000,98.000000,1.000000,-1.000000,0.000000,3.000000,0.000000
50%,40.000000,4.000000,1.000000,1.00000,0.000000,397.000000,1.000000,0.000000,0.000000,18.000000,6.000000,171.000000,2.000000,-1.000000,0.000000,3.000000,0.000000
75%,49.000000,7.000000,2.000000,2.00000,0.000000,1328.000000,1.000000,0.000000,2.000000,23.000000,8.000000,305.000000,3.000000,-1.000000,0.000000,3.000000,0.000000
max,94.000000,11.000000,2.000000,3.00000,1.000000,98417.000000,1.000000,1.000000,2.000000,31.000000,10.000000,4918.000000,63.000000,335.000000,275.000000,3.000000,1.000000


In [48]:
norm_col = ['age_in_years','balance_in_account','call_duration','days_passed']

In [49]:
from sklearn.preprocessing import scale 
X_train_scale=scale(trainset[norm_col])

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  


In [50]:
X_train_scale.shape

(31649, 4)

In [18]:
trainset['age_in_years'] = X_train_scale[:,0]

In [51]:
trainset['balance_in_account'] = X_train_scale[:,1]
trainset['call_duration'] = X_train_scale[:,2]
trainset['days_passed'] = X_train_scale[:,3]

In [52]:
X = trainset.loc[:,trainset.columns != 'outcome']
Y = trainset.loc[:,trainset.columns == 'outcome']

In [53]:
from imblearn.over_sampling import SMOTE

In [54]:
os = SMOTE(random_state=0)
columns = X.columns

In [55]:
os_data_X, os_data_Y = os.fit_sample(X,Y)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns)
os_data_Y = pd.DataFrame(data=os_data_Y,columns=['outcome'])

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [56]:
os_data_Y.shape

(59618, 1)

In [34]:
data_final_vars = trainset.columns.values.tolist()
y = ['outcome']
X = [i for i in data_final_vars if i not in y]

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(penalty='l2',C=.01)

rfe = RFE(logreg)
rfe = rfe.fit(os_data_X, os_data_Y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

[False False False  True  True False  True  True  True False  True  True
  True False False False]
[4 8 2 1 1 3 1 1 1 6 1 1 1 9 5 7]


D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
'''
select_index = np.where(rfe.ranking_ == 1)[0]
select_cols = os_data_X.columns[select_index]
train_x = os_data_X[select_cols]
train_y = os_data_Y
'''

In [57]:
X = os_data_X
y = os_data_Y

In [58]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.375636
         Iterations 7
                           Results: Logit
Model:                Logit             Pseudo R-squared:  0.458     
Dependent Variable:   outcome           AIC:               44821.3155
Date:                 2018-12-10 23:48  BIC:               44965.2469
No. Observations:     59618             Log-Likelihood:    -22395.   
Df Model:             15                LL-Null:           -41324.   
Df Residuals:         59602             LLR p-value:       0.0000    
Converged:            1.0000            Scale:             1.0000    
No. Iterations:       7.0000                                         
---------------------------------------------------------------------
                      Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
---------------------------------------------------------------------
age_in_years         -0.0438   0.0131  -3.3483 0.0008 -0.0695 -0.0182
job_description 

In [107]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)
logreg = LogisticRegression(penalty='l2',C=.01)
logreg.fit(X_train, y_train)

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [108]:
y_pred = logreg.predict(X_val)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_val, y_val)))

Accuracy of logistic regression classifier on test set: 0.84


In [109]:
s = set(y_pred)

In [110]:
s

{0, 1}

<b><i>Test dataset

In [111]:
test = pd.read_csv('Yes_Bank_Test.csv',encoding='utf-8')

In [112]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in test.columns.values:
           if test[col].dtypes=='object':
            data=test[col]
            le.fit(data.values)
            test[col]=le.transform(test[col])

In [113]:
testset = test
testset.drop('serial_number',axis=1,inplace=True)

In [114]:
X_test_scale=scale(testset[norm_col])
testset['age_in_years'] = X_test_scale[:,0]
testset['balance_in_account'] = X_test_scale[:,1]
testset['call_duration'] = X_test_scale[:,2]
testset['days_passed'] = X_test_scale[:,3]

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.


In [115]:
# testset

In [116]:
y_pred_test = logreg.predict(testset)

In [117]:
st = set(y_pred_test)
print(st)

{0, 1}


In [118]:
y_test_Sn = np.array([x for x in range(1,len(test)+1)])

In [119]:
from collections import OrderedDict 
SubDict = OrderedDict()
SubDict['serial_number'] = y_test_Sn
SubDict['outcome'] = y_pred_test

In [120]:
test_submit = pd.DataFrame(SubDict)

In [121]:
test_submit['outcome'].value_counts()

0    9886
1    3676
Name: outcome, dtype: int64

In [122]:
def deprocess_label(text):
    if text == 1:
        return 'yes'
    else:
        return 'no'

In [123]:
test_submit['outcome'] = test_submit['outcome'].apply(lambda x : deprocess_label(x))

In [124]:
test_submit.to_csv('Test_submit.csv',encoding='utf-8',index=False)

<b><i>Test on another algorithm

In [125]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(X_train,y_train)

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [126]:
y_pred = clf.predict(X_val)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(clf.score(X_val, y_val)))

Accuracy of logistic regression classifier on test set: 0.90


In [127]:
y_pred_test = clf.predict(testset)

In [128]:
from collections import OrderedDict 
SubDict = OrderedDict()
SubDict['serial_number'] = y_test_Sn
SubDict['outcome'] = y_pred_test

test_submit = pd.DataFrame(SubDict)

In [131]:
test_submit['outcome'].value_counts()

0    6961
1    6601
Name: outcome, dtype: int64

In [132]:
def deprocess_label(text):
    if text == 1:
        return 'yes'
    else:
        return 'no'

In [133]:
test_submit['outcome'] = test_submit['outcome'].apply(lambda x : deprocess_label(x))

In [134]:
test_submit.to_csv('Test_submit.csv',encoding='utf-8',index=False)

<b><i>Forest

In [86]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=None,random_state=0)
clf.fit(X_train,y_train)

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [87]:
y_pred = clf.predict(X_val)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(clf.score(X_val, y_val)))

Accuracy of logistic regression classifier on test set: 0.97


In [88]:
y_pred_test = clf.predict(testset)
from collections import OrderedDict 
SubDict = OrderedDict()
SubDict['serial_number'] = y_test_Sn
SubDict['outcome'] = y_pred_test

test_submit = pd.DataFrame(SubDict)

In [89]:
test_submit['outcome'].value_counts()

0    12064
1     1498
Name: outcome, dtype: int64

In [129]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [130]:
y_pred = clf.predict(X_val)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(clf.score(X_val, y_val)))

Accuracy of logistic regression classifier on test set: 0.95
